In [1]:
{-# LANGUAGE NegativeLiterals #-}
{-# LANGUAGE FlexibleContexts #-}

In [2]:
import Text.Parsec 
import Text.ParserCombinators.Parsec.Number
import Data.List (partition, intersect, sortBy, groupBy, sort, group, (\\))
import qualified Data.Set as S
import Data.Function (on)

In [3]:
import Debug.Trace

In [4]:
data Program = Program String Int [String]
                deriving (Show, Eq)

name (Program n _ _) = n 
weight (Program _ w _) = w
supports (Program _ _ s) = s

In [52]:
data Tree = Tree Program [Tree] Int deriving (Show, Eq)
root (Tree p _ _) = p
branches (Tree _ b _) = b
tWeight (Tree _ _ w) = w

In [6]:
onlySpaces = many (oneOf " \t")
parens = between (string "(") (string ")")
sym = many lower
commaSep sym = sym `sepBy` (onlySpaces *> string "," *> onlySpaces)

In [19]:
mFile = mLine `sepBy` newline 
mLine = Program <$> sym <*> (onlySpaces *> (parens int)) <*> supportsP
supportsP = (onlySpaces *> (string "->") *> onlySpaces *> (commaSep sym)) <|> (pure [])

In [20]:
parseFile :: String -> Either ParseError [Program]
parseFile input = parse mFile "(unknown)" input

parseLine :: String -> Either ParseError Program
parseLine input = parse mLine "(unknown)" input

successfulParse :: Either ParseError [a] -> [a]
successfulParse (Left _) = []
successfulParse (Right a) = a

In [21]:
parseLine "kuvqhnm (77)"

Right (Program "kuvqhnm" 77 [])

In [22]:
parseLine "dihjv (2158) -> gausx, ncdmp, hozgrub"

Right (Program "dihjv" 2158 ["gausx","ncdmp","hozgrub"])

In [38]:
sampleT = "pbga (66)\nxhth (57)\nebii (61)\nhavc (66)\nktlj (57)\nfwft (72) -> ktlj, cntj, xhth\nqoyq (66)\npadx (45) -> pbga, havc, qoyq\ntknk (41) -> ugml, padx, fwft\njptl (61)\nugml (68) -> gyxo, ebii, jptl\ngyxo (61)\ncntj (57)"

In [39]:
-- sample = "pbga (66)\nxhth (57)"

In [40]:
print sampleT

"pbga (66)\nxhth (57)\nebii (61)\nhavc (66)\nktlj (57)\nfwft (72) -> ktlj, cntj, xhth\nqoyq (66)\npadx (45) -> pbga, havc, qoyq\ntknk (41) -> ugml, padx, fwft\njptl (61)\nugml (68) -> gyxo, ebii, jptl\ngyxo (61)\ncntj (57)"

In [41]:
sample = successfulParse $ parseFile sampleT
sample

[Program "pbga" 66 [],Program "xhth" 57 [],Program "ebii" 61 [],Program "havc" 66 [],Program "ktlj" 57 [],Program "fwft" 72 ["ktlj","cntj","xhth"],Program "qoyq" 66 [],Program "padx" 45 ["pbga","havc","qoyq"],Program "tknk" 41 ["ugml","padx","fwft"],Program "jptl" 61 [],Program "ugml" 68 ["gyxo","ebii","jptl"],Program "gyxo" 61 [],Program "cntj" 57 []]

In [27]:
allPrograms :: [Program] -> S.Set String
allPrograms = S.fromList . map name

In [42]:
pr = allPrograms sample
pr

fromList ["cntj","ebii","fwft","gyxo","havc","jptl","ktlj","padx","pbga","qoyq","tknk","ugml","xhth"]

In [29]:
supported :: [Program] -> S.Set String
supported = S.unions . map (S.fromList . supports)

In [43]:
su = supported sample
su

fromList ["cntj","ebii","fwft","gyxo","havc","jptl","ktlj","padx","pbga","qoyq","ugml","xhth"]

In [44]:
print $ head $ S.elems $ S.difference pr su

"tknk"

In [33]:
part1 :: [Program] -> String
part1 progs = head $ S.elems $ S.difference pr su
    where su = supported progs
          pr = allPrograms progs

In [34]:
main :: IO ()
main = do 
        text <- readFile "../../data/advent07.txt"
        let progs = successfulParse $ parseFile text
        print $ part1 progs

In [35]:
main

"vtzay"

In [36]:
enTree :: Program -> Tree
enTree program = Tree program [] (weight program)

In [37]:
leaves :: [Program] -> [Program]
leaves = filter (null . supports)

In [45]:
map enTree (leaves sample)

[Tree (Program "pbga" 66 []) [] 66,Tree (Program "xhth" 57 []) [] 57,Tree (Program "ebii" 61 []) [] 61,Tree (Program "havc" 66 []) [] 66,Tree (Program "ktlj" 57 []) [] 57,Tree (Program "qoyq" 66 []) [] 66,Tree (Program "jptl" 61 []) [] 61,Tree (Program "gyxo" 61 []) [] 61,Tree (Program "cntj" 57 []) [] 57]

In [46]:
forestNames :: [Tree] -> [String]
forestNames = map (name . root)

In [47]:
forestNames $ map enTree $ leaves sample

["pbga","xhth","ebii","havc","ktlj","qoyq","jptl","gyxo","cntj"]

In [48]:
head $ filter (\p -> name p == "ugml") sample

Program "ugml" 68 ["gyxo","ebii","jptl"]

In [50]:
sampleLeaves = leaves sample
sampleBranch = sample \\ sampleLeaves

In [51]:
sampleBranch

[Program "fwft" 72 ["ktlj","cntj","xhth"],Program "padx" 45 ["pbga","havc","qoyq"],Program "tknk" 41 ["ugml","padx","fwft"],Program "ugml" 68 ["gyxo","ebii","jptl"]]

In [65]:
forest0 = map enTree sampleLeaves

In [63]:
forest0

[Tree (Program "pbga" 66 []) [] 66,Tree (Program "xhth" 57 []) [] 57,Tree (Program "ebii" 61 []) [] 61,Tree (Program "havc" 66 []) [] 66,Tree (Program "ktlj" 57 []) [] 57,Tree (Program "qoyq" 66 []) [] 66,Tree (Program "jptl" 61 []) [] 61,Tree (Program "gyxo" 61 []) [] 61,Tree (Program "cntj" 57 []) [] 57]

In [66]:
canBuild :: Program -> [Tree] -> Bool
canBuild program trees = all (\p -> p `elem` roots) $ supports program
    where roots = map (name . root) trees

In [67]:
canBuild (head sampleBranch) forest0

<interactive>:1:30: error:
    • Couldn't match type ‘Ghci51.Tree’
                     with ‘Tree’
      NB: ‘Tree’ is defined at <interactive>:1:1-55
          ‘Ghci51.Tree’ is defined at <interactive>:1:1-55
      Expected type: [Tree]
        Actual type: [Ghci51.Tree]
    • In the second argument of ‘canBuild’, namely ‘forest0’
      In the expression: canBuild (head sampleBranch) forest0
      In an equation for ‘it’: it = canBuild (head sampleBranch) forest0